# Explorando tendencias de mercado de Airbnb

La ciudad de Nueva York tiene una variedad de anuncios de Airbnb para satisfacer la alta demanda de alojamiento temporal para viajeros, con diferentes niveles de precios, tipos de habitaciones y ubicaciones.

La ciudad de Nueva York, es una de las ciudades más visitadas del mundo. Hay muchos anuncios de Airbnb en la ciudad de Nueva York para satisfacer la alta demanda de alojamiento temporal para viajeros, que puede durar desde unas pocas noches hasta muchos meses. En este proyecto, analizaremos más de cerca el mercado de Airbnb de Nueva York combinando datos de varios tipos de archivos como .csv, .tsv y .xlsx.

Recuerde que los archivos CSV, TSV y Excel son tres formatos comunes para almacenar datos. Tiene a su disposición [tres archivos](https://tajamar365.sharepoint.com/:f:/s/3405-MasterIA2024-2025/EoXMgjAJLLNJqWDyKDig8kABSUkpWj_HGQjl267qNnHS9g?e=foxjFc) que contienen datos sobre los anuncios de Airbnb de 2019:  

- `airbnb_price.csv `: Este es un archivo CSV que contiene datos sobre precios y ubicaciones de anuncios de Airbnb.  
  1. `listing_id`: identificador único del anuncio  
  2. `price`: precio del anuncio por noche en USD  
  3. `nbhood_full`: nombre del distrito y del barrio donde se encuentra el anuncio  
    

- `airbnb_room_type.xlsx`: Este es un archivo Excel que contiene datos sobre las descripciones de los anuncios de Airbnb y los tipos de habitaciones.  
  1. `listing_id`: identificador único del anuncio  
  2. `description`: descripción del anuncio     
  3. `room_type`: Airbnb tiene tres tipos de habitaciones: habitaciones compartidas, habitaciones privadas y casas o apartamentos completos 


- `airbnb_last_review.tsv`: Este es un archivo TSV que contiene datos sobre los nombres de los anfitriones de Airbnb y las fechas de revisión.  
  1. `listing_id`: identificador único del anuncio  
  2. `host_name`:  nombre del anfitrión del anuncio  
  3. `last_review`: echa en la que se revisó el anuncio por última vez   

Como consultor que trabaja para una empresa inmobiliaria emergente, ha recopilado datos de anuncios de Airbnb de varias fuentes para investigar el mercado de alquileres a corto plazo en Nueva York. Analizará estos datos para brindar información sobre habitaciones privadas a la empresa inmobiliaria.

- ¿Cuáles son las fechas de las primeras y las últimas reseñas? Almacene estos valores como dos variables independientes con los nombres que prefiera.
- ¿Cuántos de los anuncios son habitaciones privadas? Guárdelo en cualquier variable.
- ¿Cuál es el precio promedio de los anuncios? Redondee a los dos decimales más cercanos y guárdelo en una variable.
- Combine las nuevas variables en un DataFrame llamado review_dates con cuatro columnas en el siguiente orden: first_reviewed, last_reviewed, nb_private_rooms y avg_price. El DataFrame solo debe contener una fila de valores.

Preparativos

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, min, max, count, avg, round, to_date

spark = SparkSession.builder.appName("AirbnbDataProcessing").getOrCreate()

dfp=spark.read.load(path="dbfs:/FileStore/shared_uploads/sergio.ferreras@tajamar365.com/airbnb_price.csv", format="csv", header="true", sep=",", inferSchema="true")

dft=spark.read.load(path="dbfs:/FileStore/shared_uploads/sergio.ferreras@tajamar365.com/airbnb_room_type.csv", format="csv", header="true", sep=";", inferSchema="true")

dfr = spark.read.load(path="dbfs:/FileStore/shared_uploads/sergio.ferreras@tajamar365.com/airbnb_last_review.tsv", format="csv", header="true", sep="\t", inferSchema="true")


¿Cuáles son las fechas de las primeras y las últimas reseñas? Almacene estos valores como dos variables independientes con los nombres que prefiera.

In [0]:
from pyspark.sql import functions as F


dfr = dfr.withColumn("last_review_date", F.to_date(F.col("last_review"), "MMMM dd yyyy"))

reseñaantigua = dfr.orderBy("last_review_date", ascending=True).first()
reseñaantigua = reseñaantigua["last_review"]

reseñanueva = dfr.orderBy("last_review_date", ascending=False).first()
reseñanueva = reseñanueva["last_review"]

print("Reseña más antigua:")
print(reseñaantigua)
print("Reseña más reciente:")
print(reseñanueva)

Reseña más antigua:
January 01 2019
Reseña más reciente:
July 09 2019


¿Cuántos de los anuncios son habitaciones privadas? Guárdelo en cualquier variable.

In [0]:
#filtramos las filas cogiendo las que tengan como tipo de habitacion Private Room
npr = dft.filter(dft.room_type == "Private room").count()

print("Habitaciones Privadas:")
print(npr)

Habitaciones Privadas:
7185


¿Cuál es el precio promedio de los anuncios? Redondee a los dos decimales más cercanos y guárdelo en una variable.

In [0]:
from pyspark.sql.functions import regexp_replace

#primero hacemos una limpieza de los datos para eliminar todo lo que  no sean numeros
dfp = dfp.withColumn(
    "price", 
    regexp_replace("price", "[a-zA-Z\\s]", "")
)

#usando avg hacemos una media del precio de todas las filas de price aproximandolo a 2 decimales y lo guardamos en una variable
avg_price = dfp.select(round(avg("price"), 2).alias("avg_price")).collect()[0]["avg_price"]
print(avg_price)

141.78


Combine las nuevas variables en un DataFrame llamado review_dates con cuatro columnas en el siguiente orden: first_reviewed, last_reviewed, nb_private_rooms y avg_price. El DataFrame solo debe contener una fila de valores.

In [0]:
from pyspark.sql import Row

#creamos una fila en un nuevo dataframe usando Row donde ponemos los datos de las variable anteriores  
review_data = Row(
    first_reviewed=reseñaantigua,  
    last_reviewed=reseñanueva,   
    nb_private_rooms=npr,         
    avg_price=avg_price           
)

review_dates = spark.createDataFrame([review_data])

review_dates.display()

first_reviewed,last_reviewed,nb_private_rooms,avg_price
January 01 2019,July 09 2019,7185,141.78
